In [2]:
import pygame as pg
import random
from time import sleep
import time 

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
BLACK= ( 0,  0,  0)
WHITE= (255,255,255)
BLUE = ( 0,  0, 255)
GREEN= ( 0, 255,  0)
RED  = (255,  0,  0)

size = [1024, 512]
background_width = 1024
bat_width = 110
bat_height = 67
aircraft_width = 90
aircraft_height = 55

fireball1_width = 140
fireball1_height = 60
fireball2_width = 86
fireball2_height = 60

def textObj(text, font):
    textSurface = font.render(text, True, RED)
    return textSurface, textSurface.get_rect()

def printtext(text):
    global screen
    font = pg.font.Font('freesansbold.ttf', 30)
    text = font.render(str(text), True, BLACK)
    screen.blit(text,(size[0]/2,size[1]/2))
    
def dispMessage(text):
    global screen
    
    largeText = pg.font.Font('freesansbold.ttf', 115)
    TextSurf, TextRect = textObj(text, largeText)
    TextRect.center = ((size[0]/2),(size[1]/2))
    screen.blit(TextSurf, TextRect)
    pg.display.update()
    sleep(2)
    runGame()
    
def crash():
    global screen
    dispMessage('Dead!')

def back(background, x, y):
    global screen
    screen.blit(background, (x,y))

def airplane(x, y):
    global screen, aircraft
    screen.blit(aircraft,(x,y))
    
def drawObject(obj, x, y):
    global screen
    screen.blit(obj,(x,y))

def runGame():
    global screen, clock, aircraft, background1, background2
    global bat, fires, bullet, boom, start_time
    
    isShotBat = False
    boom_count = 0
    
    bullet_xy = []
    
    x = size[0] * 0.05
    y = size[1] * 0.8
    y_change = 0
    
    background1_x = 0
    background2_x = background_width
    
    bat_x = size[0]
    bat_y = random.randrange(0,size[1])
    
    fire_x = size[0]
    fire_y = random.randrange(0, size[1])
    random.shuffle(fires)
    fire = fires[0]
    
    done = False
    while not done:
        
        clock.tick(60)

        for event in pg.event.get(): 
            if event.type == pg.QUIT: 
                done = True 
            
            if event.type == pg.KEYDOWN:
                if event.key == pg.K_UP:
                    y_change = -5
                elif event.key == pg.K_DOWN:
                    y_change = 5
                    
                elif event.key == pg.K_LCTRL:
                    bullet_x = x + aircraft_width
                    bullet_y = y + aircraft_height / 2
                    bullet_xy.append([bullet_x, bullet_y])
                elif event.key == pg.K_SPACE:
                    sleep(5)
                    
            if event.type == pg.KEYUP:
                if event.key == pg.K_UP or event.key == pg.K_DOWN:
                    y_change = 0
        
        # game screen
        screen.fill(WHITE)
        
        # Aircraft 
        y += y_change
        if y < 0 :
            y = 0
        elif y > size[1] - aircraft_height:
            y = size[1] - aircraft_height
        
        # Background
        background1_x -= 2
        background2_x -= 2
        
        if background1_x == -background_width:
            background1_x = background_width
            
        if background2_x == -background_width:
            background2_x = background_width
            
        back(background1, background1_x, 0)
        back(background2, background2_x, 0)
        
        # Bat
        bat_x -= 7
        
        if bat_x <= 0:
            bat_x = size[0]
            bat_y = random.randrange(0,size[1])
            
        
        # Fireball
        if fire[1] == None:
            fire_x -= 30
        else:
            fire_x -= 15
            
        if fire_x <= 0:
            fire_x = size[0]
            fire_y = random.randrange(0,size[1])
            random.shuffle(fires)
            fire = fires[0]
        
        # Bullet
        if len(bullet_xy) != 0:
            for i, bxy in enumerate(bullet_xy):
                bxy[0] += 15
                bullet_xy[i][0] = bxy[0]
                
                if bxy[0] > bat_x:
                    if bxy[1] > bat_y and bxy[1] < bat_y + bat_height:
                        bullet_xy.remove(bxy)
                        isShotBat = True
                        
                if bxy[0] >= size[0]:
                    try:
                        bullet_xy.remove(bxy)
                    except:
                        pass
        
        if x + aircraft_width > bat_x:
            if (y > bat_y and y < bat_y + bat_height) or (y + aircraft_height > bat_y and y + aircraft_height < bat_y + bat_height):
                crash()
                
        if fire[1] != None:
            if fire[0] == 0:
                fireball_width = fireball1_width
                fireball_height = fireball1_height
            elif fire[0] == 1:
                fireball_width = fireball2_width
                fireball_height = fireball2_height
            if x + aircraft_width > fire_x:
                if (y > fire_y and y < fire_y + fireball_height) or (y + aircraft_height > fire_y and y + aircraft_height < fire_y + fireball_height):
                    crash()
   
        if fire[1] != None:
            drawObject(fire[1], fire_x, fire_y)
        
        if len(bullet_xy) != 0:
            for bx, by in bullet_xy:
                drawObject(bullet, bx, by)
        
        # Boom
        if not isShotBat:
            drawObject(bat, bat_x,bat_y)
        else:
            drawObject(boom, bat_x, bat_y)
            boom_count += 1
            if boom_count > 5:
                boom_count = 0
                bat_x = size[0]
                bat_y = random.randrange(0, size[1] - bat_height)
                isShotBat = False
        
        
        
        airplane(x,y)
        pg.display.flip()
        
    pg.quit()
    quit()
    
def initGame():
    global screen, clock, aircraft, background1, background2
    global bat, fires, bullet, boom, start_time
    
    fires = []
    
    pg.init()
    screen = pg.display.set_mode(size)
    pg.display.set_caption("Sukwoo's Game")
    aircraft = pg.image.load('./pygame/plane.png')
    background1 = pg.image.load('./pygame/background.jpg')
    background2 = background1.copy()
    bat = pg.image.load('./pygame/bat.png')
    fires.append((0,pg.image.load('./pygame/fireball.png')))
    fires.append((1,pg.image.load('./pygame/fireball2.png')))
    
    boom = pg.image.load('./pygame/boom.png')
    
    for i in range(5):
        fires.append((i+2,None))
        
    bullet = pg.image.load('./pygame/bullet.png')
    clock = pg.time.Clock()
    start_time = pg.time.get_ticks()
    runGame()
    
initGame()